In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

ModuleNotFoundError: No module named 'dm4bem'

### Introduction

In this first part we will define our working environment. The plan of our house can be found below. The objective of this study case is the nalysis of a cubic building with 5 identical walls & a transparent wall (glass window), air infiltration, and HVAC system controlling the indoor air temperature.

### Design model

The room that we will study is presented in the figure below. Two rooms are separated by a small wall without insulation. The total area of the model is about 9 meters square and the ceiling is 3 meters high. The rooms have each 3 exterior walls and one interior wall. We will consider the house as insulated from the outside .
The interior walls are made of concrete. The room also has a 1 m² window, a 2 m² door and a HVAC device.



<center><img src="https://hub.gke2.mybinder.org/user/cghiaus-dm4bem-19y1e1gf/files/Sch%C3%A9ma%20maison.png?_xsrf=2%7C3ce45c22%7Cea41507d6477cef712308b01cb5ff2db%7C1682345682"></center>

> Figure 1. Plan of the building



We will now define boundary conditions wich are important to solve our case.


### Hypothesis

Boundary Conditions:

- Temperature of the exterior air = 0 °C
- Setpoint temperature of the interior air = 18 °C
- Unique temperature on each wall
- Unidirectionnal heat transfer

### Dimension of windows, doors, and walls.

In [3]:
l = 3               # m length of the cubic room
Sg = l**2           # m² surface of the glass wall
Sc = Si = 5 * Sg    # m² surface of concrete & insulation of the 5 walls
Sc1=2*Sg
Sc2=3*Sg

Thermo-physical properties
The thermophysical properties of the air (in SI units) are:

In [2]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat in': 1000,
        'Specific heat out' : 25}               # J/(kg·K)
# pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

NameError: name 'pd' is not defined

We use concrete, glass and insulation (glass wool) as materials. We take our door with oak wood. 

In [1]:
BETON = {'Conductivity': 1.400,
            'Density': 2400.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': 60}

ISOLANT = {'Conductivity': 0.046,
              'Density': 25.0,
              'Specific heat': 1030,
              'Width': 0.08,
              'Surface':45}

VERRE = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': 40}

PORTE = {'Conductivity': 0.15,
         'Density': 800,
         'Specific heat': 2000,
         'Width': 0.1,
         'Surface': 4}

wall = pd.DataFrame.from_dict({'Layer_out (insulation)': ISOLANT,
                               'Layer_in (concrete)': BETON,
                               'Glass': VERRE,
                               'Door': PORTE},
                              orient='index')


NameError: name 'pd' is not defined

### Thermal circuit

We will now able to draw our thermal circuit. The objective with this one will be to computerize our model. The figure will be found below.

<center><img src="https://hub.gke2.mybinder.org/user/cghiaus-dm4bem-19y1e1gf/files/Sch%C3%A9ma%20%C3%A9lec.png?_xsrf=2%7C3ce45c22%7Cea41507d6477cef712308b01cb5ff2db%7C1682345682"></center>

>Figure 2. Thermal circuit


### Convection coefficients

Conventional values for the [convection coeficients](https://energieplus-lesite.be/theories/enveloppe9/echanges-chaleur-parois/resistance-thermique-d-echange-superficiel/) for indoor and outdoor convection in W/(m²⋅K) are:

In [ ]:
h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)
# h = pd.DataFrame([{'in': 8., 'out': 25}])  # W/(m²⋅K)


### Thermal coductances
#### Conduction
The conductances 1, 2, 3, and 4 of the thermal circuit from Figure 3 model the heat transfer by [conduction](https://en.m.wikipedia.org/wiki/Thermal_conduction). Conduction conductances, in W/K, are of the form:
$$G_{cd} = \frac{\lambda}{w}S$$
where:

- $\lambda$ - [thermal conductvity](https://en.m.wikipedia.org/wiki/Thermal_conductivity), W/(m⋅K);
- $w$ - width of the material, m;
- $S$ - surface area of the wall, m².

In [5]:
# conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns={'Conductance'})

NameError: name 'wall' is not defined

#### Convection
The conductances 0, 6 and 7 model the heat transfer by [convection]. Convection conductances, in W/K, are of the form:
$$G_{cv} = {h S}$$
where:
- $h$ is the [convection coefficient], W/(m²⋅K);
- $S$ - surface area of the wall, m². 

In [ ]:
# convection
Gw = h * wall['Surface'][0]     # wall
Gg = h * wall['Surface'][2]     # glass

### Partie codage 

Création du code 

$$\left\{\begin{array}{ll}
C \dot{\theta} = -(A^T G A) \theta + A^T G b + f\\ 
q = G (-A \theta + b)
\end{array}\right.$$

In [5]:
#Initial Values#

Text = 0                        #°C
Tvent = 18                      #°C
Tint1 = 20                      #°C
Tint2 = 20                      #°C
lambda_concrete = 1.4
lambda_insulation = 0.046
lambda_glass = 1.4
lambda_wood = 0.15
hi = 8
ho = 25


Swall = 9
Swindow = 1
Sdoor = 2
Sint = Swall*8 - Swindow - Sdoor

import numpy as np
A= np.zeros((24,17))
A[0, 0] = 1
A[1, 1] = 1
A[1, 0] = -1
A[2, 2] = 1
A[2, 1] = -1
A[3, 3] = 1
A[3, 2] = -1
A[4, 4] = 1
A[4, 3] = -1
A[6, 6] = 1
A[6, 4] = -1
A[5, 5] = 1
A[5, 4] = -1
A[8, 7] = 1
A[9, 5] = 1
A[9, 7] = -1
A[7, 6] = 1
A[7, 5] = -1
A[10, 6] = 1
A[11, 6] = 1
A[12, 6] = 1
A[12, 8] = -1
A[13, 8] = 1
A[13, 9] = -1
A[14, 16] = 1
A[14, 10] = -1
A[15, 10] = 1
A[15, 11] = -1
A[16, 11] = 1
A[16, 12] = -1
A[17, 12] = 1
A[17, 13] = -1
A[18, 13] = 1
A[19, 15] = 1
A[19, 14] = -1
A[20, 14] = 1
A[21, 9] = 1
A[23, 9] = 1
A[23, 16] = -1
A[22, 9] = 1
A[22, 15] = -1

B=np.zeros(24)
B[0]=1
B[8]=1
B[10]=1
B[11]=1
B[18]=1
B[20]=1
B[21]=1

G=np.zeros((24,24))
G[0,0]=air['Specific heat in']*Sc1
G[1,1]=BETON['Width']/(BETON['Conductivity'])*Sc1
G[2,2]=BETON['Width']/(BETON['Conductivity'])*Sc1
G[3,3]=ISOLANT['Width']/(ISOLANT['Conductivity'])*Sc1
G[4,4]=ISOLANT['Width']/(ISOLANT['Conductivity'])*Sc1
G[5,5]=air['Specific heat out']*Sc1
G[6,6]=air['Specific heat out']*Sc1
G[7,7]=air['Specific heat out']*Sg
G[8,8]=air['Specific heat in']*Sg
G[9,9]=VERRE['Width']/(VERRE['Conductivity'])*Sg
G[10,10]=air['Specific heat in']*Sc1
G[11,11]=air['Specific heat out']*Sg
G[12,12]=BETON['Width']/(BETON['Conductivity'])*Sg
G[13,13]=BETON['Width']/(BETON['Conductivity'])*Sg
G[14,14]=ISOLANT['Width']/(ISOLANT['Conductivity'])*Sc2
G[15,15]=ISOLANT['Width']/(ISOLANT['Conductivity'])*Sc2
G[16,16]=BETON['Width']/(BETON['Conductivity'])*Sc2
G[17,17]=BETON['Width']/(BETON['Conductivity'])*Sc2
G[18,18]=air['Specific heat out']*Sc2
G[19,19]=PORTE['Width']/(PORTE['Conductivity'])*PORTE['Surface']
G[20,20]=air['Specific heat in']*Sc2
G[21,21]=air['Specific heat out']*Sc2
G[22,22]=air['Specific heat out']*Sc2
G[23,23]=air['Specific heat out']*Sc2

C=np.zeros((17,17))
C[1,1]=BETON['Density']*BETON['Width']*BETON['Specific heat']*Sc1
C[1,1]=ISOLANT['Density']*ISOLANT['Width']*ISOLANT['Specific heat']*Sc1
C[1,1]=BETON['Density']*BETON['Width']*BETON['Specific heat']*Sg
C[1,1]=BETON['Density']*BETON['Width']*BETON['Specific heat']*Sc2
C[1,1]=ISOLANT['Density']*ISOLANT['Width']*ISOLANT['Specific heat']*Sc2

# Vector of temperature sources
# =============================


f = np.zeros(17)
f[0] = 1
f[4] = 1
f[6] = 1
f[7] = 1
f[9] = 1
f[13] = 1


b = np.zeros(24)
b[0] = 1
b[8] = 1
b[10] = 1
b[11] = 1
b[18] = 1
b[20] = 1
b[21] = 1

y = np.zeros(16)         # nodes
y[[9]] = 1              # nodes (temperatures) of interest


[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, b, C, f, y)
print('As = \n', As, '\n')
print('Bs = \n', Bs, '\n')
print('Cs = \n', Cs, '\n')
print('Ds = \n', Ds, '\n')

u = np.hstack([b, f])
print(f'u = {u}')

yss = (-Cs @ np.linalg.inv(As) @ Bs + Ds) @ u
print(f'yss = {yss} °C')

λ = np.linalg.eig(As)[0]    # eigenvalues of matrix As

print('Time constants: \n', -1 / λ, 's \n')
print('2 x Time constants: \n', -2 / λ, 's \n')
dtmax = 2 * min(-1. / λ)
print(f'Maximum time step: {dtmax:.2f} s = {dtmax / 60:.2f} min')

# time step
dt = np.floor(dtmax / 60) * 60   # s
print(f'dt = {dt} s = {dt / 60:.0f} min')


# settling time
time_const = np.array([int(x) for x in sorted(-1 / λ)])
print('4 * Time constants: \n', 4 * time_const, 's \n')

t_settle = 4 * max(-1 / λ)
print(f'Settling time: \
{t_settle:.0f} s = \
{t_settle / 60:.1f} min = \
{t_settle / (3600):.2f} h = \
{t_settle / (3600 * 24):.2f} days')

# Step response
# -------------
# Find the next multiple of 3600 s that is larger than t_settle
duration = np.ceil(t_settle / 3600) * 3600
n = int(np.floor(duration / dt))    # number of time steps
t = np.arange(0, n * dt, dt)        # time vector for n time steps

print(f'Duration = {duration} s')
print(f'Number of time steps = {n}')
# pd.DataFrame(t, columns=['time'])

u = np.zeros([17, n])                # u = [To To To Tisp Φo Φi Qa Φa]
u[0:3, :] = 10 * np.ones([3, n])    # To = 10 for n time steps
u[3, :] = 20 * np.ones([1, n])      # Tisp = 20 for n time steps

# pd.DataFrame(u)

n_s = As.shape[0]                      # number of state variables
θ_exp = np.zeros([n_s, t.shape[0]])    # explicit Euler in time t
θ_imp = np.zeros([n_s, t.shape[0]])    # implicit Euler in time t

I = np.eye(n_s)                        # identity matrix

for k in range(n - 1):
    θ_exp[:, k + 1] = (I + dt * As) @\
        θ_exp[:, k] + dt * Bs @ u[:, k]
    θ_imp[:, k + 1] = np.linalg.inv(I - dt * As) @\
        (θ_imp[:, k] + dt * Bs @ u[:, k])
    
    
y_exp = Cs @ θ_exp + Ds @  u
y_imp = Cs @ θ_imp + Ds @  u

fig, ax = plt.subplots()
ax.plot(t / 3600, y_exp.T, t / 3600, y_imp.T)
ax.set(xlabel='Time, $t$ / h',
       ylabel='Temperatue, $θ_i$ / °C',
       title='Step input: outdoor temperature $T_o$')
ax.legend(['Explicit', 'Implicit'])
ax.grid()
plt.show()



NameError: name 'air' is not defined